In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None

        

In [12]:
### Import everything needed to edit/save/watch video clips
import imageio
import pickle
from moviepy.editor import VideoFileClip
from IPython.display import HTML
# Set up lines for left and right
left_lane = Line()
right_lane = Line()

#mtx, dist = getCameraCallibration()
dist_pickle = pickle.load( open( "wide_dist_pickle.p", "rb" ) )
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

white_output = 'white.mp4'
clip1 = VideoFileClip("project_video.mp4")

white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)


[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [02:13<00:00,  9.63it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

Wall time: 2min 14s


In [3]:
# This function will color the image
# Input: Original image
# Output: Original image with colored region
def process_image(image):

    src = np.float32(
            [[120, 720],
             [550, 470],
             [750, 470],#700 470
             [1160, 720]])

    dst = np.float32(
            [[200,720],
             [200,0],
             [1080,0],
             [1080,720]])

        
    result = None
    # Apply pipeline to the image to create black and white image
    combined_binary = pipeline(image)

    binary_warped = image_PerspectiveTransform(combined_binary, src,dst)

    findLanes(binary_warped)
    
    #Draw final Lane
    if sanityCheck(left_lane, right_lane):
        findLanes(binary_warped, scheck=True)
    
    result = image_reverse_PerspectiveTransform(image, binary_warped,left_lane.recent_xfitted,right_lane.recent_xfitted,src,dst)

    return result

In [4]:
def pipeline(img, s_thresh=(170, 205), sx_thresh=(20, 100)):
    
    result = None
    
    #Get the undistorted image
    img = cv2.undistort(img, mtx, dist, None, mtx)
    
    # Convert to HSV color space and separate the V channel
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HLS).astype(np.float)
    l_channel = hsv[:,:,1]
    s_channel = hsv[:,:,2]
    
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))
    
    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1

    
    return combined_binary


In [5]:
def findLanes(binary_warped, nwindows=9, scheck=False):
    
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 200
    
    leftx_current, rightx_current = None, None
    
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    if scheck == True:
        right_lane.detected = False
        left_lane.detected = False
        scheck = False
    
    
    if ((left_lane.detected == False and right_lane.detected == False) and scheck == False):
        histogram = np.sum(binary_warped[np.int(binary_warped.shape[0]/2):,:], axis=0)    
        midpoint = np.int(histogram.shape[0]/2) + 160
        
        if right_lane.bestx != None and scheck == False:
            shift_right = int(right_lane.bestx)
            leftx_base = np.argmax(histogram[:midpoint])
            rightx_base = np.argmax(histogram[shift_right:]) + shift_right
        else:
            
            leftx_base = np.argmax(histogram[:midpoint])
            rightx_base = np.argmax(histogram[midpoint:]) + midpoint
            
        # Current positions to be updated for each window
        leftx_current = leftx_base
        rightx_current = rightx_base
    else:
        leftx_current = int(left_lane.bestx)
        rightx_current = int(right_lane.bestx)


    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 
    
    
    
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])

    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)


    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    left_curverad, right_curverad = findCurvetures(ploty, leftx, lefty, rightx, righty)
    base_position = int(np.mean(left_fitx) + ((np.mean(right_fitx) - np.mean(left_fitx))/2))
    
    
    #Build the left Line object
    left_lane.detected = False
    # x values of the last n fits of the line
    left_lane.recent_xfitted = left_fitx
    #average x values of the fitted line over the last n iterations
    left_lane.bestx = np.mean(left_fitx)     
    #polynomial coefficients averaged over the last n iterations
    left_lane.best_fit = np.mean(left_fit)
    #polynomial coefficients for the most recent fit
    left_lane.current_fit = left_fit[0]
    #radius of curvature of the line in some units
    left_lane.radius_of_curvature = left_curverad
    #distance in meters of vehicle center from the line
    left_lane.line_base_pos = base_position - left_lane.bestx 
    #difference in fit coefficients between last and new fits
    #self.diffs = np.array([0,0,0], dtype='float') 
    #x values for detected line pixels
    left_lane.allx = leftx  
    #y values for detected line pixels
    left_lane.ally = lefty

    #Build the right Line object
    right_lane.detected = False  
    # x values of the last n fits of the line
    right_lane.recent_xfitted = right_fitx
    #average x values of the fitted line over the last n iterations
    right_lane.bestx = np.mean(right_fitx)     
    #polynomial coefficients averaged over the last n iterations
    right_lane.best_fit = np.mean(right_fit)
    #polynomial coefficients for the most recent fit
    right_lane.current_fit = right_fit[0]
    #radius of curvature of the line in some units
    right_lane.radius_of_curvature = right_curverad
    #distance in meters of vehicle center from the line
    right_lane.line_base_pos = right_lane.bestx - base_position
    #difference in fit coefficients between last and new fits
    #self.diffs = np.array([0,0,0], dtype='float') 
    #x values for detected line pixels
    right_lane.allx = rightx  
    #y values for detected line pixels
    right_lane.ally = righty
    
    return right_lane, right_lane

In [6]:
def sanityCheck(left_lane, right_lane):
    
    result = True
    bot_lane_fit_diff = int(right_lane.recent_xfitted[len(right_lane.recent_xfitted)-1] - left_lane.recent_xfitted[len(left_lane.recent_xfitted)-1])
    top_lane_fit_diff = int(right_lane.recent_xfitted[0] - left_lane.recent_xfitted[0])
    
    if ((top_lane_fit_diff > 550 and top_lane_fit_diff < 790) and (bot_lane_fit_diff > 610 and bot_lane_fit_diff < 800)):
        left_lane.detected = True  
        right_lane.detected = True  
        result = False
    return result

In [7]:
def findCurvetures(ploty,leftx, lefty,rightx, righty):
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    y_eval = np.max(ploty)
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
 
    return left_curverad, right_curverad

In [8]:
def image_PerspectiveTransform(color_binary, src,dst):

    M = cv2.getPerspectiveTransform(src,dst)
    warped = cv2.warpPerspective(color_binary, M, color_binary.shape[::-1], flags=cv2.INTER_LINEAR)   

    binary_warped = np.zeros_like(warped)
    binary_warped[(warped == 1)] = 1
    
    return binary_warped

In [9]:
def image_reverse_PerspectiveTransform(image, binary_warped, left_fitx, right_fitx, src, dst):
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    Minv = cv2.getPerspectiveTransform(dst,src)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    
    # Combine the result with the original image
    result = cv2.addWeighted(image, 1, newwarp, 0.3, 0)

    font = cv2.FONT_HERSHEY_SIMPLEX
    text = "Radius of Curvature: {} m".format(int(left_lane.radius_of_curvature))
    cv2.putText(result,text,(400,100), font, 1,(255,255,255),2)
    # Find the position of the car
    xm_per_pix = 3.7/700
    position = binary_warped.shape[1]/2
    pts = np.argwhere(newwarp[:,:,1])
    left  = np.min(pts[(pts[:,1] < position) & (pts[:,0] > 700)][:,1])
    right = np.max(pts[(pts[:,1] > position) & (pts[:,0] > 700)][:,1])
    center = (left + right)/2
    
    positionx = (position - center)*xm_per_pix
    
    if positionx < 0:
        text = "Vehicle is {:.2f} m left of center".format(-positionx)
    else:
        text = "Vehicle is {:.2f} m right of center".format(positionx)
    cv2.putText(result,text,(400,150), font, 1,(255,255,255),2)    
    
    return result

In [10]:
def getCameraCallibration():
    
    nx = 9 # The number of inside corners in x
    ny = 6 # The number of inside corners in y
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((nx*ny,3), np.float32)
    objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')
    
    #Get the image size
    img = cv2.imread(images[0])
    img_size = (img.shape[1], img.shape[0])
    
    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)
        #print(ret)
        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)    
    
    

    # Do camera calibration given object points and image points
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)

    dist_pickle = {}
    dist_pickle["mtx"] = mtx
    dist_pickle["dist"] = dist
    pickle.dump( dist_pickle, open( "wide_dist_pickle.p", "wb" ) )
    
    return mtx, dist

In [11]:
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    
    abs_sobel = None
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    grad_binary = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    grad_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1    
    
    return grad_binary

def mag_thresh(image, sobel_kernel=3, mag_thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    mag_binary = np.zeros_like(gradmag)
    mag_binary[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    
    return mag_binary

def dir_threshold(image, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    dir_binary =  np.zeros_like(absgraddir)
    dir_binary[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
    
    return dir_binary